# Importing dataset

In [1]:
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import os
import io
import pandas as pd
import cv2
import sklearn
import random
from osgeo import gdal, osr
import shutil

# PATHS

In [2]:
DATA_PATH = "D:/cbers_data/DataSetModelo/"
RAW_DATA_DIR = "D:/cbers_data/DataSetModelo/raw_scene/"
RAW_MASK_DIR = "D:/cbers_data/DataSetModelo/raw_mask/"
CLIPPED_DATA_DIR = "D:/cbers_data/DataSetModelo/clip/"
VRT_DATA_DIR = "D:/cbers_data/DataSetModelo/vrt/"
PREDICT_DIR = "D:/cbers_data/DataSetModelo/predict/"

# Calibrating Model

# Utils

In [3]:
def normalize_min_max(image):
    
    return((image-np.nanmin(image))/(np.nanmax(image)- np.nanmin(image)))

def resize_img(image,x,y,z):
    image = cv2.resize(image,(x,y))
    image = image.reshape((x,y,z))
    return image

def print_cbers_image(image):
    plt.figure()
    plt.imshow(image)
    plt.show()

# HyperParameters

In [5]:
DIMENSION = 256
BANDS = 3
BATCH_SIZE = 4

# Pre processing the raw imagery

In [4]:
DATA_PATH = "D:/cbers_data/DataSetModelo/CALIBRATING_MODEL/"
TRAINING_PATH = "D:/cbers_data/DataSetModelo/CALIBRATING_MODEL/Training/"
SCENES_DIR = "D:/cbers_data/DataSetModelo/CALIBRATING_MODEL/Scenes/"
MASKS_DIR = "D:/cbers_data/DataSetModelo/CALIBRATING_MODEL/Masks/"
NORMALIZED_DIR = "D:/cbers_data/DataSetModelo/CALIBRATING_MODEL/Normalized_scenes/"
VRT_DATA_DIR = "D:/cbers_data/DataSetModelo/CALIBRATING_MODEL/vrt/"

In [ ]:
from utils.handle_data import create_training_folders

In [ ]:
#Create training folders
create_training_folders(TRAINING_PATH)

## Transform raw data to clipped image (Necessário quando as imagens estão separadas)

In [ ]:
#Get all files from raw_data
FILES = os.listdir(SCENES_DIR)
#get name of files
FILES = [i.split('_BAND', 1)[0] for i in FILES]
#remove duplicateds
FILES = list(dict.fromkeys(FILES))
#Clip All images first
for image in FILES:
    clip_image(SCENES_DIR,CLIPPED_DATA_DIR,image)

# Normalize Scene

In [6]:
RAW_SCENES_LIST = os.listdir(SCENES_DIR)

In [7]:
from utils.normalize impo

In [ ]:
driver

In [ ]:
scene.min()

In [ ]:
scene.max()

In [ ]:
raster_Y_size

In [ ]:
    r = driver.Create(filename, t.RasterXSize, t.RasterYSize, 3, gdal.GDT_Byte,['COMPRESS=LZW'])

    # Set metadata
    r.SetGeoTransform(t.GetGeoTransform())
    r.SetProjection(t.GetProjection())

    # loop through bands and write new values
    for bix in range(3):

        rb = r.GetRasterBand(bix+1)

        # Write array
        rb.WriteArray(arr[...,bix])


In [ ]:
#Split dataset files
all_frames = os.listdir(CLIPPED_DATA_DIR)
random.seed(12)
random.shuffle(all_frames)

train_size = int(0.7 * len(all_frames))
val_size = int(0.25 * len(all_frames))
test_size = int(0.5 * len(all_frames))


train_scene = all_frames[:train_size]
val_scene = all_frames[train_size:train_size+val_size]
test_scene = val_subscenes = all_frames[train_size+val_size:]

In [ ]:
# Generate corresponding mask lists for masks
train_masks = [f for f in all_frames if f in train_scene]
val_masks = [f for f in all_frames if f in val_scene]
test_masks = [f for f in all_frames if f in test_scene]


In [ ]:
def crop_image_rgbn(output_dir, rgbn, image_name, dimension):
    gt = rgbn.GetGeoTransform()
    x_min = gt[0]
    y_max = gt[3]

    res = gt[1]

    num_img_x = rgbn.RasterXSize/dimension
    num_img_y = rgbn.RasterYSize/dimension

    x_len = res * rgbn.RasterXSize
    y_len = res * rgbn.RasterYSize

    x_size = x_len/num_img_x
    y_size = y_len/num_img_y


    x_steps = [x_min + x_size * i for i in range(int(num_img_x) + 1)]
    y_steps = [y_max - y_size * i for i in range(int(num_img_y) + 1)]
    print("qnt img x: " + str(num_img_x))
    print("qnt img y: " + str(num_img_y))
    index_img = 0
    for i in range(int(num_img_x)):
        for j in range(int(num_img_y)):
            x_min = x_steps[i]
            x_max = x_steps[i+1]
            y_max = y_steps[j]
            y_min = y_steps[j+1]
            index_img+=1
            print(output_dir + image_name+ "_" + str(i)+ "_" +str(j)+'_'+str(index_img)+"_"+".tif")
            gdal.Warp(output_dir + image_name+ "_" + str(i)+ "_" +str(j)+'_'+str(index_img)+"_"+".tif", rgbn, 
                      outputBounds = (x_min,y_min,x_max, y_max), dstNodata = -9999)

In [ ]:
#Add train, val frames and masks to relevant folders
from numpy import save

def add_frames(dir_name, image_name):
    try:
        image_name = image_name.split('_rgbn')[0]
        output_dir = DATA_PATH + dir_name + '/'
        #crop image
        clipped_image = gdal.Open(CLIPPED_DATA_DIR + image_name +'_rgbn.tif')
        
        crop_image_rgbn(output_dir, clipped_image, image_name, DIMENSION)
        
    except Exception as e: 
        #print(e)
        pass
    
def add_masks(dir_name, image_name):
    try:
        image_name = image_name.split('_rgbn')[0]
        dir_name = DATA_PATH + dir_name + '/'
        #crop image
        mask = gdal.Open(RAW_MASK_DIR + image_name + '_CMASK_GRID_SURFACE.tif')
        crop_image_rgbn(dir_name, mask, image_name, DIMENSION)
 
    except Exception as e:
        #print(e)
        pass

In [ ]:
frame_folders = [(train_scene, 'train_frames'), (val_scene, 'val_frames'), (test_scene, 'test_frames')]

mask_folders = [(train_masks, 'train_masks'), (val_masks, 'val_masks'), (test_masks, 'test_masks')]

In [ ]:
#Celula que carrega as imagens nos diretórios corretos

from PIL import Image

# Add masks
for folder in mask_folders:
    array = folder[0]
    name = [folder[1]] * len(array)
    list(map(add_masks, name, array))

# Add frames
for folder in frame_folders:
    array = folder[0]
    name = [folder[1]] * len(array)
    list(map(add_frames, name, array))




# Creating the generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        data_format=None,
        )

In [ ]:
BATCH_SIZE = 4

train_image_generator = train_datagen.flow_from_directory(
'D:/cbers_data/DataSetModelo/train_frames',
batch_size = BATCH_SIZE)


In [ ]:
import cv2
import random
from sklearn.model_selection import ParameterGrid

In [ ]:
from sklearn import preprocessing
def data_gen(img_folder, mask_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    random.shuffle(n)
    x,y,z = DIMENSION,DIMENSION,1
    while(True):
        img = np.zeros((batch_size, DIMENSION, DIMENSION, 3)).astype('float')
        mask = np.zeros((batch_size, DIMENSION, DIMENSION, 1)).astype('float')

        for i in range(c, c + batch_size): #initially from 0 to 16, c = 0. 
            #train_img = np.load(img_folder+'/'+n[i], allow_pickle=True)
            train_img = gdal.Open(img_folder+'/'+n[i]).ReadAsArray()

            #train_img =  cv2.resize(train_img, (256, 256))# Read an image from folder and resize
            #train_img = train_img.reshape(256, 256, 4)
            #Normalização
            train_img = cv2.normalize(train_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

            #train_img = normalize_min_max(train_img)
            train_img = tf.convert_to_tensor(train_img.transpose((1,2,0)))
            img[i-c] = train_img #add to array - img[0], img[1], and so on.

            #n[i] = n[i][:-3] + "tif"
            
            train_mask = gdal.Open(mask_folder+'/'+n[i]).ReadAsArray()
            #train_mask = (np.load(mask_folder+'/'+n[i]))
            train_mask = train_mask/1
            #train_mask = cv2.resize(train_mask, (256, 256))
            train_mask = resize_img(train_mask,x,y,z)
            #train_mask = normalize_min_max(train_mask)
            train_mask = cv2.normalize(train_mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            #train_mask = np.expand_dims(train_mask,axis=2)
            train_mask = np.expand_dims(train_mask, axis=2)
            train_mask = tf.convert_to_tensor(train_mask)
            
            mask[i-c] = train_mask
            

        c += batch_size
        
        if(c + batch_size >= len(os.listdir(img_folder))):
            c=0
            random.shuffle(n)
                      # print "randomizing again"
        
        yield img, mask
        
train_frame_path = 'D:/cbers_data/DataSetModelo/train_frames'
train_mask_path = 'D:/cbers_data/DataSetModelo/train_masks'

val_frame_path = 'D:/cbers_data/DataSetModelo/val_frames'
val_mask_path = 'D:/cbers_data/DataSetModelo/val_masks'

test_frame_path = 'D:/cbers_data/DataSetModelo/test_frames'
test_mask_path = 'D:/cbers_data/DataSetModelo/test_masks'

train_gen = data_gen(train_frame_path,train_mask_path, batch_size = BATCH_SIZE)
val_gen = data_gen(val_frame_path,val_mask_path, batch_size = BATCH_SIZE)
test_gen = data_gen(test_frame_path,test_mask_path, batch_size = BATCH_SIZE)

# F1 SCORE

# Creating the model

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
NO_OF_TRAINING_IMAGES = len(os.listdir(train_frame_path))
NO_OF_VAL_IMAGES = len(os.listdir(val_frame_path))
weights_path = 'D:/cbers_data/DataSetModelo/'

checkpoint = ModelCheckpoint(weights_path, monitor='f1', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('D:/cbers_data/DataSetModelo/log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor = 'f1', verbose = 1,
                              min_delta = 0.01, patience = 5, mode = 'max')

callbacks_list = [checkpoint, csv_logger, earlystopping]




In [ ]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

from itertools import chain
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

In [ ]:
# Build U-Net model

inputs = Input((DIMENSION, DIMENSION, 3))
s = (inputs)

c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy",f1])
model.summary()

In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, ZeroPadding2D
from collections import defaultdict


visualkeras.layered_view(model,legend=True, scale_xy=0.5, scale_z=0.5, max_z=7, to_file='network.png')

### Training

In [ ]:
#training the model
model_history = model.fit(train_gen, 
                    epochs = 15, 
                    batch_size = BATCH_SIZE, 
                    validation_data = val_gen, 
                    callbacks = callbacks_list,
                    steps_per_epoch =(NO_OF_TRAINING_IMAGES/BATCH_SIZE), 
                    validation_steps = (NO_OF_VAL_IMAGES/BATCH_SIZE)
                  )

model.save('model')

In [ ]:
from IPython.display import Image, display
import PIL
from PIL import ImageOps


def get_test_batch(qnt_images):
    evaluate = []
    
    convert_to_image = keras.preprocessing.image.array_to_img
    dir_images = test_frame_path
    dir_masks = test_mask_path
    test_files = os.listdir(test_frame_path)
    #batch_images_structure
    mask_true_image_test = np.zeros((qnt_images,DIMENSION,DIMENSION,1))
    batch_test_images = np.zeros((qnt_images,DIMENSION,DIMENSION,3))

    for i in range(qnt_images):
        mask_test = gdal.Open(dir_masks + '/'+ test_files[i]).ReadAsArray()
        mask_test = cv2.normalize(mask_test, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        mask_test = np.expand_dims(mask_test, axis=2)

        imagem_teste = gdal.Open(dir_images + '/'+ test_files[i]).ReadAsArray()
        imagem_teste = imagem_teste.transpose((1,2,0))
        imagem_teste = cv2.normalize(imagem_teste, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        
        y_eval = np.expand_dims(mask_test, axis=0)
        x_eval = np.expand_dims(imagem_teste, axis=0)
        
        evaluate.append(model.evaluate(x=x_eval,y=y_eval))
        
        mask_true_image_test [i,:,:] = mask_test
        batch_test_images [i,:,:] = imagem_teste
    return batch_test_images, mask_true_image_test, evaluate

In [ ]:
qnt_images = 500
batch_test_images, mask_true_image_test, evaluate = get_test_batch(qnt_images)

In [ ]:
df_eval = pd.DataFrame(evaluate)
df_eval.columns = ["Loss", "Accuracy", "f1"]
df_eval

In [ ]:
print( 'Loss: ' + str(df_eval['Loss'].mean()))
print( 'F1: ' + str(df_eval['f1'].mean()))
print( 'Accuracy: ' + str(df_eval['Accuracy'].mean()))

In [ ]:
mask_predict = model.predict(batch_test_images)

In [ ]:
def show_images(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'CMask', 'Predicted Mask']

    
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
        
    plt.show()

In [ ]:
for i in range(qnt_images):
    b, g, r  = batch_test_images[i][:, :, 0], batch_test_images[i][:, :, 1], batch_test_images[i][:, :, 2]
    rgb = np.dstack((r,g,b))
    show_images([rgb, mask_true_image_test[i], mask_predict[i]])

# Aprendizado

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']

loss = model_history.history['loss']
val_loss = model_history.history['val_loss']


plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.show()

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']


plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='lower right')
plt.ylabel('Loss')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Loss')

plt.show()

In [ ]:
f1 = model_history.history['f1']
val_f1 = model_history.history['val_f1']


plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(f1, label='Training F1-Score')
plt.plot(val_f1, label='Validation F1-Score')
plt.legend(loc='lower right')
plt.ylabel('F1-Score')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation F1-Score')

plt.show()

In [ ]:
import glob
croped = 'D:/cbers_data/DataSetModelo/train_frames/'
list_image = glob.glob(croped + "CBERS_4A_WFI_20201020_205_116_L4_*_*_*_.tif")
sorted(list_image, key=lambda x: int(x.split('_')[11]))
g = gdal.Warp("output.tif", list_image)

# MODELO JA TREINADO

# Recarregamento do modelo treinado

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import os
import io
import pandas as pd
import cv2
import sklearn
import random
from osgeo import gdal
import shutil

In [ ]:
DATA_PATH = "D:/cbers_data/DataSetModelo/"
RAW_DATA_DIR = "D:/cbers_data/DataSetModelo/raw_scene/"
RAW_MASK_DIR = "D:/cbers_data/DataSetModelo/raw_mask/"
CLIPPED_DATA_DIR = "D:/cbers_data/DataSetModelo/clip/"
VRT_DATA_DIR = "D:/cbers_data/DataSetModelo/vrt/"
PREDICT_DIR = "D:/cbers_data/DataSetModelo/predict/"
DIMENSION = 256

# Functions

In [ ]:
def normalize_min_max(image):
    
    return((image-np.nanmin(image))/(np.nanmax(image)- np.nanmin(image)))

def resize_img(image,x,y,z):
    image = cv2.resize(image,(x,y))
    image = image.reshape((x,y,z))
    return image

def print_cbers_image(image):
    plt.figure()
    plt.imshow(image)
    plt.show()

In [ ]:
def clip_image(input_path, output_path,image_name):
    red = gdal.Open(input_path + image_name + '_BAND13_GRID_SURFACE.tif')
    green = gdal.Open(input_path + image_name + '_BAND14_GRID_SURFACE.tif')
    blue = gdal.Open(input_path + image_name + '_BAND15_GRID_SURFACE.tif')
    #nir = gdal.Open(input_path + image_name + '_BAND16_GRID_SURFACE.tif')
    #Create the .vrt from RGBN
    #array = [red, green, blue,/nir]#adicionar o nir
    array = [red, green, blue]#adicionar o nir
    opt = gdal.BuildVRTOptions(srcNodata=-9999, VRTNodata=-9999,separate=True,resampleAlg='nearest')
    vrt_clip = gdal.BuildVRT(VRT_DATA_DIR + image_name +'.vrt', array, options=opt)
    #Translate the .vrt to .tif
    trans_opt = gdal.TranslateOptions(format="tif", outputType=gdal.gdalconst.GDT_Unknown, 
                                  bandList=[1,2,3],width=0, height=0, widthPct=0.0, 
                                  heightPct=0.0, xRes=0.0, yRes=0.0,noData=-9999)
    #Clip da imagem
    gdal.Translate(output_path + image_name +'_rgbn.tif', vrt_clip)
    #Apaga o vrt
    #os.remove(image_name + ".vrt") 
    return True
    

In [ ]:
def crop_image_rgbn(output_dir, rgbn, image_name, dimension):
    gt = rgbn.GetGeoTransform()
    x_min = gt[0]
    y_max = gt[3]

    res = gt[1]

    num_img_x = rgbn.RasterXSize/dimension
    num_img_y = rgbn.RasterYSize/dimension

    x_len = res * rgbn.RasterXSize
    y_len = res * rgbn.RasterYSize

    x_size = x_len/num_img_x
    y_size = y_len/num_img_y


    x_steps = [x_min + x_size * i for i in range(int(num_img_x) + 1)]
    y_steps = [y_max - y_size * i for i in range(int(num_img_y) + 1)]
    print("qnt img x: " + str(num_img_x))
    print("qnt img y: " + str(num_img_y))
    index_img = 0
    for i in range(int(num_img_x)):
        for j in range(int(num_img_y)):
            x_min = x_steps[i]
            x_max = x_steps[i+1]
            y_max = y_steps[j]
            y_min = y_steps[j+1]
            index_img+=1
            
            gdal.Warp(output_dir + image_name+ "_" + str(i)+ "_" +str(j)+'_'+str(index_img)+"_"+".tif", rgbn, 
                      outputBounds = (x_min,y_min,x_max, y_max), dstNodata = -9999)

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Load model and predict

In [ ]:
import glob
TESTE_FRAME_DIR = 'D:/cbers_data/DataSetModelo/test_frames/'
TESTE_MASK_DIR = 'D:/cbers_data/DataSetModelo/test_masks/'
# Load image

test_frames_list = glob.glob(TESTE_FRAME_DIR + "CBERS_4A_WFI_20201025_235_108_L4_*_*_*_.tif")
test_masks_list = glob.glob(TESTE_MASK_DIR + "CBERS_4A_WFI_20201025_235_108_L4_*_*_*_.tif")

In [ ]:
QNT_SUB_FRAMES = len(test_frames_list)
test_images = np.zeros((QNT_SUB_FRAMES,DIMENSION,DIMENSION,3))

for i, ele in enumerate(test_frames_list):
    image_test = gdal.Open(ele).ReadAsArray()
    image_test = image_test.transpose((1,2,0))
    image_test = cv2.normalize(image_test, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    test_images [i,:,:] = image_test
    plt.imshow(tf.keras.preprocessing.image.array_to_img(image_test))
    plt.show()

In [ ]:
def createRGB(template,arr,filename):
    '''Creates a copy of a 3-band raster with values from array

    Arguments:

        template: Path to template raster
        arr: Value array with dimensions (r,c,3)
        filename: Output filename for new raster 
    '''

    # Open template
    t = gdal.Open(template)

    # Get geotiff driver
    driver = gdal.GetDriverByName('GTiff')

    # Create new raster
    r = driver.Create(filename, t.RasterXSize, t.RasterYSize, 3, gdal.GDT_Byte,['COMPRESS=LZW'])

    # Set metadata
    r.SetGeoTransform(t.GetGeoTransform())
    r.SetProjection(t.GetProjection())

    # loop through bands and write new values
    for bix in range(3):

        rb = r.GetRasterBand(bix+1)

        # Write array
        rb.WriteArray(arr[...,bix])

    # Close datasets
    t = None
    r = None
    rb = None

In [ ]:
model = tf.keras.models.load_model('model', custom_objects={'f1':f1})

In [ ]:
test_predicts = model.predict(test_images)

In [ ]:
#MOSTRA AS MASCARAS DAS IMAGENS
def show_mask(test_predicts):
    plt.figure(figsize=(15, 15))

    title = 'Predicted Mask'
    plt.title(title)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(test_predicts))
    plt.axis('off')
        
    plt.show()

In [ ]:
#TRANSFORMA MASCARA OUTPUT EM RASTER
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Byte)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()


In [ ]:
for i, file_input in enumerate(test_frames_list):
    file_output = file_input.replace('test_frames','predict')
    aux = gdal.Open(file_input)
    geo_t = aux.GetGeoTransform()
    origin = (geo_t[0],geo_t[3])
    px_width = geo_t[1]
    px_height = geo_t[5]
    array = test_predicts[i,:, :, 0]
    array2raster(file_output,origin,px_width,px_height,array)

In [ ]:
for i, file_input in enumerate(test_frames_list):
    file_output = file_input.replace('test_frames','predict')

In [ ]:
qnt_images = 500
for i in range(qnt_images):
    b, g, r  = batch_test_images[i][:, :, 0], batch_test_images[i][:, :, 1], batch_test_images[i][:, :, 2]
    rgb = np.dstack((r,g,b))
    show_images([rgb, mask_true_image_test[i], mask_predict[i]])

In [ ]:
for i in range(500):
    plt.figure(figsize=(15, 15))
    show_images([test_predicts[i]])

In [ ]:
test_predicts[321]*1000

In [ ]:
teste = gdal.Open('D:\\cbers_data\\DataSetModelo\\test_masks\\CBERS_4A_WFI_20201020_205_116_L4_0_51_52_.tif').ReadAsArray()
teste = np.expand_dims(teste,2)
show_mask(teste)

# Production

In [ ]:
#dimensoes subscenes
DIMENSION = 256
BATCH_SIZE = 4

In [ ]:
#CAMINHO DOS ARQUIVOS DE ENTRADA (IMAGEM COMPLETA) (ENTRADA)
production_frame_path = "D:/cbers_data/DataSetModelo/ProductionFrames/"

#CAMINHO QUE ARMAZENARÁ AS SUBCENAS DA IMAGEM DE ENTRADA (SAIDA)
production_cropped_frame_path = "D:/cbers_data/DataSetModelo/ProductionCroppedFrames/"

In [ ]:
#Função responsável por cortar as imagens de entrada e salvá-las num diretório de saída
def save_cropped_production_images(input_path_base, output_path_base, DIMENSION):
    #Pega lista de imagens a serem cortadas
    raw_images_list_name = os.listdir(production_frame_path)
    for frame_name in raw_images_list_name:
        #Pasta de saída terá mesmo nome do arquivo de entrada (porém sem o .tif)
        output_path = output_path_base + frame_name.split('.')[0]+"/"
        #Caminho dos arquivos de entrada (imagens cruas)
        input_path = input_path_base + frame_name
        #Abre a imagem de entrada
        raw_scene = gdal.Open(input_path)
        #Cria uma pasta com o nome do arquivo de entrada
        if not os.path.isdir(output_path):
            os.mkdir(output_path)
        #Corta as imagens de entrada e as salva no diretório de saída
        crop_image_rgbn(output_path, raw_scene, frame_name.split(".")[0], DIMENSION)
        
#Chamada da função
save_cropped_production_images(production_frame_path,production_cropped_frame_path, DIMENSION)

In [ ]:
#LISTA QUE CONTEM O CAMINHO DE CADA SUBCENA GERADA (ENTRADAS)
subscenes_path_list = [production_cropped_frame_path + file_name + "/" for file_name in os.listdir(production_cropped_frame_path)]

#CAMINHO QUE ARMAZENARÁ AS MASCARAS DAS SUBCENAS (SAIDA)
production_cropped_mask_path = "D:/cbers_data/DataSetModelo/ProductionCroppedMasks/"

In [ ]:
#Função responsável por chamar o modelo para gerar as mascaras e a salvar no diretório de saída
def save_production_cropped_masks(input_dir_list, output_path):
    #FOR que caminha entre os diretórios das subcenas
    for i, dir_name in enumerate(input_dir_list):
        #FOR que entrou no diretório de uma subcenas e caminha por elas
        input_file_list = os.listdir(dir_name)
        for j, file_name in enumerate(input_file_list):
            subscene = gdal.Open(dir_name+file_name)
            subscene_array = subscene.ReadAsArray().transpose((1,2,0))
            subscene_normalized =  cv2.normalize(subscene_array, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            #Dimensões são adequadas para o predict do modelo
            subscene_expanded = np.expand_dims(subscene_normalized, axis=0)
            subscene_mask = model.predict(subscene_expanded)
            
            ##Etapa para configurar o .tif das mascaras
            driver = gdal.GetDriverByName("GTiff")
            metadata = driver.GetMetadata()
            
            #Transforma shape da mascara de (1,256,256,1) para (256,256)
            subscene_mask = subscene_mask.squeeze()
            #NOME DA CENA ORIGINAL (IMAGEM INTEIRA)
            original_scene_name = file_name.split("rgbn")[0]+"rgbn"
            #Cria uma pasta com o nome do arquivo original (IMAGEM COMPLETA)
            if not os.path.isdir(output_path + original_scene_name):
                os.mkdir(output_path + original_scene_name)
            #Cria um raster (.tif) da mascara com o mesmo tamanho da subscena
            subscene_mask_raster = driver.Create(output_path + original_scene_name + "/" + file_name,
                                subscene.RasterXSize,
                                subscene.RasterYSize,
                                1, #Numero de bandas da mascara
                                gdal.GDT_Float32)
            #COPIA AS PROJEÇÕES DA SUBSCENA
            subscene_mask_raster.SetProjection(subscene.GetProjectionRef())
            subscene_mask_raster.SetGeoTransform(subscene.GetGeoTransform()) 
            
            #Get the band to write to
            #out_band = subscene.GetRasterBand(1)
            #ESCREVE A MASCARA EM DISCO
            subscene_mask_raster.WriteArray(subscene_mask)
            
save_production_cropped_masks(subscenes_path_list, production_cropped_mask_path)

In [ ]:
#LISTA QUE CONTEM O CAMINHO DE CADA MASCARA GERADA (ENTRADAS)
masks_path_list = [production_cropped_mask_path + file_name + "/" for file_name in os.listdir(production_cropped_mask_path)]


# Caminho para o diretório onde deverá ser salvo a máscara após o mosaico (SAIDA)
mask_mosaic_path = "D:/cbers_data/DataSetModelo/MergeFile/"

In [ ]:
for i, folder_name in enumerate(masks_path_list):
    folder_input_list = os.listdir(folder_name)
    files_to_mosaic_path = [folder_name + file_name  for file_name in folder_input_list]
    print(folder_input_list[0].split('rgbn')[0] + "rgbn/")
            

In [ ]:
import osgeo_utils.gdal_merge
def mount_mosaic_of_masks(input_path, output_path):
    for i, folder_name in enumerate(input_path):
        files_to_mosaic = os.listdir(folder_name)
        files_to_mosaic_path = [folder_name + file_name  for file_name in files_to_mosaic]
        #Cria uma pasta com o nome do arquivo original (IMAGEM COMPLETA)
        original_file_name = files_to_mosaic[0].split("rgbn")[0]+"rgbn"
        if not os.path.isdir(output_path + original_file_name):
            os.mkdir(output_path + original_file_name)
        #ESCREVE EM DISCO
        print(output_path + original_file_name + "/" + original_file_name + 
                      ".tif")
        g = gdal.Warp(output_path + original_file_name + "/" + original_file_name + 
                      ".tif", files_to_mosaic_path, format="GTiff")
        g = None # Close file and flush to disk
        
mount_mosaic_of_masks(masks_path_list, mask_mosaic_path)